In [10]:
%pip install tensorflowjs

  Using cached tensorflowjs-4.22.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached flax-0.12.1-py3-none-any.whl.metadata (11 kB)
  Using cached tf_keras-2.20.1-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_decision_forests-1.8.1-cp311-cp311-win_amd64.whl
  Using cached tensorflow_hub-0.16.1-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached jax-0.8.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.8.1-cp311-cp311-win_amd64.whl.metadata (1.4 kB)
  Using cached ml_dtypes-0.5.4-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
  Using cached numpy-2.3.5-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.12.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached tensorflow-2.15.1-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached tensorflow_intel-2.15.1-cp311-cp311-win_amd64.whl.metadata (4.9 kB)
INFO: pip is l

ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'c:\\Users\\molka\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\quantization\\tensorflow\\python\\pywrap_quantize_model.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install scikit-learn-porter

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement scikit-learn-porter (from versions: none)
ERROR: No matching distribution found for scikit-learn-porter

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import joblib
import numpy as np
import json

In [3]:
lgb_model = joblib.load('lgb_model.pkl')

In [4]:

def extract_detailed_model_info(model):
    """Extrait des informations détaillées du modèle LightGBM"""
    
    info = {
        "feature_names": [
            'ip_exist', 'abnormal_url', 'dot_count', 'www_count', '@_count', 
            'hyphen_count', 'subdomain_count', 'shortening_service', 'https_count', 
            'http_count', 'percent_count', 'query_count', 'equal_count', 
            'url_length', 'hostname_length', 'no_embed', 'suspicious_words',
            'digit_count', 'letters_count', 'fd_length', 'tld_length'
        ],
        "class_names": ['benign', 'defacement', 'malware', 'phishing'],
        "n_classes": model.n_classes_,
        "n_features": model.n_features_in_,
        "feature_importances": model.feature_importances_.tolist(),
        "boosting_type": model.boosting_type,
        "objective": model.objective,
        "n_estimators": model.n_estimators,
        "learning_rate": model.learning_rate,
        "max_depth": model.max_depth
    }
    
    # Normaliser les importances des features
    importances = np.array(info["feature_importances"])
    if importances.sum() > 0:
        info["feature_importances_normalized"] = (importances / importances.sum()).tolist()
    else:
        info["feature_importances_normalized"] = [0] * len(importances)
    
    # Extraire quelques règles simples des premiers arbres
    info["simple_rules"] = extract_simple_rules(model)
    
    return info

def extract_simple_rules(model, max_trees=10):
    """Extrait des règles simples des premiers arbres"""
    rules = []
    
    try:
        # Pour chaque classe (dans un modèle multiclasse)
        for class_idx in range(model.n_classes_):
            class_rules = []
            
            # Prendre seulement les premiers arbres pour cette classe
            trees_for_class = model.estimators_[class_idx]
            
            for tree_idx in range(min(max_trees, len(trees_for_class))):
                tree = trees_for_class[tree_idx].tree_
                
                # Extraire les features importantes de cet arbre
                feature_indices = np.unique(tree.feature[tree.feature >= 0])
                if len(feature_indices) > 0:
                    class_rules.append({
                        "tree": tree_idx,
                        "important_features": feature_indices.tolist(),
                        "thresholds": tree.threshold[tree.feature >= 0][:5].tolist()  # 5 premiers
                    })
            
            rules.append({
                "class": class_idx,
                "rules": class_rules[:5]  # 5 premières règles
            })
    except:
        # Si échec, créer des règles basées sur les feature importances
        rules = [
            {
                "class": 0,
                "rules": [{"important_features": [0, 1, 4, 7], "description": "Benign indicators"}]
            },
            {
                "class": 1,
                "rules": [{"important_features": [2, 6, 13], "description": "Defacement indicators"}]
            },
            {
                "class": 2,
                "rules": [{"important_features": [0, 1, 13, 17], "description": "Malware indicators"}]
            },
            {
                "class": 3,
                "rules": [{"important_features": [16, 7, 4, 10], "description": "Phishing indicators"}]
            }
        ]
    
    return rules

# Sauvegarder les informations détaillées
model_info = extract_detailed_model_info(lgb_model)

with open('model/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("✅ Detailed model info saved")
print(f"Feature importances: {model_info['feature_importances_normalized']}")

✅ Detailed model info saved
Feature importances: [0.00425, 0.004666666666666667, 0.0805, 0.044083333333333335, 0.005833333333333334, 0.04933333333333333, 0.10516666666666667, 0.011, 0.0215, 0.00975, 0.019083333333333334, 0.017333333333333333, 0.03325, 0.08583333333333333, 0.11966666666666667, 0.0033333333333333335, 0.021, 0.09783333333333333, 0.06641666666666667, 0.1515, 0.048666666666666664]


NameError: name 'extract_model_structure' is not defined